In [1]:
import argparse

from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import Optimizer, nihgcn
from myutils import *
from sampler import NewSampler
from sklearn.model_selection import KFold

In [3]:
class Args:
    def __init__(self):
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )  # cuda:number or cpu
        self.data = "gdsc2"  # Dataset{gdsc or ccle}
        self.lr = 0.001  # the learning rate
        self.wd = 1e-5  # the weight decay for l2 normalizaton
        self.layer_size = [1024, 1024]  # Output sizes of every layer
        self.alpha = 0.25  # the scale for balance gcn and ni
        self.gamma = 8  # the scale for sigmod
        self.epochs = 1000  # the epochs for model


args = Args()

In [4]:
res, drug_finger, exprs, null_mask, pos_num = load_data(args)
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

target_dim = [
    # 0,  # Cell
    1  # Drug
]

load gdsc2


In [5]:
def nihgcn_new(
    cell_exprs,
    drug_finger,
    res_mat,
    null_mask,
    target_dim,
    target_index,
    evaluate_fun,
    args,
):

    sampler = NewSampler(res_mat, null_mask, target_dim, target_index)
    model = nihgcn(
        sampler.train_data,
        cell_exprs=cell_exprs,
        drug_finger=drug_finger,
        layer_size=args.layer_size,
        alpha=args.alpha,
        gamma=args.gamma,
        device=args.device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        evaluate_fun,
        lr=args.lr,
        wd=args.wd,
        epochs=args.epochs,
        device=args.device,
    )
    true_data, predict_data = opt()
    return true_data, predict_data

In [ ]:
n_kfold = 1
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()
for dim in target_dim:
    for target_index in tqdm(np.arange(res.shape[dim])):
        if dim:
            if drug_sum[target_index] < 10:
                continue
        else:
            if cell_sum[target_index] < 10:
                continue
        epochs = []
        for fold in range(n_kfold):
            true_data, predict_data = nihgcn_new(
                cell_exprs=exprs,
                drug_finger=drug_finger,
                res_mat=res,
                null_mask=null_mask,
                target_dim=dim,
                target_index=target_index,
                evaluate_fun=roc_auc,
                args=args,
            )

        true_data_s = pd.concat(
            [true_data_s, translate_result(true_data)], ignore_index=True
        )
        predict_data_s = pd.concat(
            [predict_data_s, translate_result(predict_data)], ignore_index=True
        )

  0%|          | 0/240 [00:00<?, ?it/s]

epoch:   0 loss:0.709002 auc:0.4520
epoch:  20 loss:0.153209 auc:0.7782
epoch:  40 loss:0.136905 auc:0.7480
epoch:  60 loss:0.123181 auc:0.7450
epoch:  80 loss:0.114529 auc:0.7411
epoch: 100 loss:0.108614 auc:0.7862
epoch: 120 loss:0.102904 auc:0.8392
epoch: 140 loss:0.097554 auc:0.8728
epoch: 160 loss:0.093757 auc:0.8831
epoch: 180 loss:0.089240 auc:0.8947
epoch: 200 loss:0.088499 auc:0.8843
epoch: 220 loss:0.083874 auc:0.9143
epoch: 240 loss:0.081974 auc:0.9151
epoch: 260 loss:0.079845 auc:0.9091
epoch: 280 loss:0.079814 auc:0.9015
epoch: 300 loss:0.077965 auc:0.9287
epoch: 320 loss:0.076641 auc:0.9257
epoch: 340 loss:0.076715 auc:0.9249
epoch: 360 loss:0.076369 auc:0.9216
epoch: 380 loss:0.075000 auc:0.9233
epoch: 400 loss:0.074535 auc:0.9273
epoch: 420 loss:0.074820 auc:0.9299
epoch: 440 loss:0.073995 auc:0.9334
epoch: 460 loss:0.074004 auc:0.9199
epoch: 480 loss:0.073494 auc:0.9366
epoch: 500 loss:0.074390 auc:0.9322
epoch: 520 loss:0.074599 auc:0.9098
epoch: 540 loss:0.073108 auc

  2%|▏         | 4/240 [00:26<26:08,  6.64s/it]

Fit finished.
epoch:   0 loss:0.698697 auc:0.4932
epoch:  20 loss:0.153377 auc:0.7558
epoch:  40 loss:0.137354 auc:0.8113
epoch:  60 loss:0.123877 auc:0.8148
epoch:  80 loss:0.115146 auc:0.8062
epoch: 100 loss:0.109181 auc:0.7927
epoch: 120 loss:0.103416 auc:0.7716
epoch: 140 loss:0.098962 auc:0.7643
epoch: 160 loss:0.093636 auc:0.7479
epoch: 180 loss:0.091060 auc:0.6975
epoch: 200 loss:0.086107 auc:0.7798
epoch: 220 loss:0.084773 auc:0.8030
epoch: 240 loss:0.081823 auc:0.7354
epoch: 260 loss:0.080202 auc:0.7794
epoch: 280 loss:0.078778 auc:0.7577
epoch: 300 loss:0.077666 auc:0.7669
epoch: 320 loss:0.077196 auc:0.7720
epoch: 340 loss:0.076178 auc:0.7720
epoch: 360 loss:0.075723 auc:0.7613
epoch: 380 loss:0.075019 auc:0.7896
epoch: 400 loss:0.074900 auc:0.7876
epoch: 420 loss:0.074671 auc:0.8020
epoch: 440 loss:0.074495 auc:0.7952
epoch: 460 loss:0.076313 auc:0.8188
epoch: 480 loss:0.073850 auc:0.7855
epoch: 500 loss:0.073621 auc:0.7843
epoch: 520 loss:0.075731 auc:0.7240
epoch: 540 los

  5%|▍         | 11/240 [00:43<13:55,  3.65s/it]

...

Fit finished.
epoch:   0 loss:0.700229 auc:0.3008
epoch:  20 loss:0.153279 auc:0.6016
epoch:  40 loss:0.136531 auc:0.8242
epoch:  60 loss:0.122893 auc:0.8672
epoch:  80 loss:0.114492 auc:0.8789
epoch: 100 loss:0.108560 auc:0.8672
epoch: 120 loss:0.102656 auc:0.8789
epoch: 140 loss:0.097903 auc:0.8281
epoch: 160 loss:0.092697 auc:0.8789
epoch: 180 loss:0.088541 auc:0.8828
epoch: 200 loss:0.085562 auc:0.8828
epoch: 220 loss:0.084067 auc:0.8750
epoch: 240 loss:0.081320 auc:0.8867
epoch: 260 loss:0.080025 auc:0.8750
epoch: 280 loss:0.079164 auc:0.8984
epoch: 300 loss:0.077818 auc:0.8789
epoch: 320 loss:0.078005 auc:0.9062
epoch: 340 loss:0.076452 auc:0.8906
epoch: 360 loss:0.075858 auc:0.8984
epoch: 380 loss:0.075553 auc:0.8984
epoch: 400 loss:0.075876 auc:0.8984
epoch: 420 loss:0.074733 auc:0.8984
epoch: 440 loss:0.080384 auc:0.7656
epoch: 460 loss:0.074690 auc:0.8984
epoch: 480 loss:0.074111 auc:0.8828
epoch: 500 loss:0.073780 auc:0.8828
epoch: 520 loss:0.074963 auc:0.8828
epoch: 540 los

  8%|▊         | 19/240 [02:27<45:33, 12.37s/it]

Fit finished.
epoch:   0 loss:0.703923 auc:0.5062
epoch:  20 loss:0.152641 auc:0.5793
epoch:  40 loss:0.136324 auc:0.6101
epoch:  60 loss:0.122515 auc:0.6202
epoch:  80 loss:0.113876 auc:0.6026
epoch: 100 loss:0.107965 auc:0.5857
epoch: 120 loss:0.102169 auc:0.5831
epoch: 140 loss:0.097301 auc:0.5821
epoch: 160 loss:0.092345 auc:0.5830
epoch: 180 loss:0.092155 auc:0.5678
epoch: 200 loss:0.085766 auc:0.5742
epoch: 220 loss:0.083082 auc:0.5805
epoch: 240 loss:0.081278 auc:0.5830
epoch: 260 loss:0.080000 auc:0.5910
epoch: 280 loss:0.078452 auc:0.5866
epoch: 300 loss:0.077874 auc:0.5994
epoch: 320 loss:0.076450 auc:0.6020
epoch: 340 loss:0.077538 auc:0.6056
epoch: 360 loss:0.075334 auc:0.6087
epoch: 380 loss:0.074794 auc:0.6110
epoch: 400 loss:0.074322 auc:0.6115
epoch: 420 loss:0.076283 auc:0.6049
epoch: 440 loss:0.073896 auc:0.6114
epoch: 460 loss:0.073848 auc:0.6116
epoch: 480 loss:0.073404 auc:0.6124
epoch: 500 loss:0.073405 auc:0.6149
epoch: 520 loss:0.073883 auc:0.6073
epoch: 540 los

  8%|▊         | 20/240 [02:44<49:36, 13.53s/it]

Fit finished.
epoch:   0 loss:0.705128 auc:0.4577
epoch:  20 loss:0.154089 auc:0.7366
epoch:  40 loss:0.138420 auc:0.5450
epoch:  60 loss:0.124269 auc:0.5031
epoch:  80 loss:0.114637 auc:0.5484
epoch: 100 loss:0.108424 auc:0.6075
epoch: 120 loss:0.102501 auc:0.6567
epoch: 140 loss:0.096847 auc:0.7073
epoch: 160 loss:0.092945 auc:0.7332
epoch: 180 loss:0.088590 auc:0.7396
epoch: 200 loss:0.085412 auc:0.7473
epoch: 220 loss:0.085187 auc:0.7338
epoch: 240 loss:0.081475 auc:0.7587
epoch: 260 loss:0.080264 auc:0.7598
epoch: 280 loss:0.078513 auc:0.7600
epoch: 300 loss:0.078355 auc:0.7475
epoch: 320 loss:0.076717 auc:0.7491
epoch: 340 loss:0.079981 auc:0.7649
epoch: 360 loss:0.075818 auc:0.7457
epoch: 380 loss:0.075912 auc:0.7258
epoch: 400 loss:0.074566 auc:0.7340
epoch: 420 loss:0.074749 auc:0.7441
epoch: 440 loss:0.074563 auc:0.7394
epoch: 460 loss:0.075190 auc:0.7396
epoch: 480 loss:0.073618 auc:0.7401
epoch: 500 loss:0.073914 auc:0.7310
epoch: 520 loss:0.074115 auc:0.7386
epoch: 540 los

In [ ]:
true_data_s.to_csv(f"new_drug_true_{args.data}.csv")
predict_data_s.to_csv(f"new_drug_pred_{args.data}.csv")